За нулевую гипотезу примем отсутствие разницы в конверсии из двух источников.

Параметр alpha выберем 0.05.

Загрузим необходимые библиотеки и данные

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
from scipy import stats as st
import statsmodels.api as sm

import statsmodels.stats.proportion as proportion
import statsmodels.stats.power as power
import statsmodels.stats.gof as gof

df = pd.read_csv('datasets/visits.csv')

In [13]:
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17521 entries, 0 to 17520
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   uid                17521 non-null  int64 
 1   date               17521 non-null  object
 2   source             17521 non-null  object
 3   is_send_complaint  17521 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 547.7+ KB


,uid,date,source,is_send_complaint
0,6124,2021-04-01 00:01:55,Источник1,0
1,13047,2021-04-01 00:02:59,Источник2,0
2,7613,2021-04-01 00:14:34,Источник1,1
3,7034,2021-04-01 00:16:05,Источник1,0
4,3810,2021-04-01 00:30:07,Источник1,0
...,...,...,...,...
17516,4895,2021-04-29 23:48:44,Источник1,1
17517,12605,2021-04-29 23:50:48,Источник2,0
17518,4640,2021-04-29 23:55:37,Источник1,0
17519,9091,2021-04-29 23:57:30,Источник2,1


Данные выглядят нормально, пропусков не обнаруджено.

Проверим не дублируются ли uid.

In [3]:
df.uid.value_counts(sort=True)

2047     1
6790     1
10880    1
8833     1
14978    1
        ..
13596    1
3355     1
1306     1
7449     1
0        1
Name: uid, Length: 17521, dtype: int64

Подготовим необходимые переменные

In [4]:
s1 = df.query('source == "Источник1"').is_send_complaint.sum()      #  число успехов Источник 1
n1 = df.query('source == "Источник1"').is_send_complaint.count()    #  число испытаний Источник 1
s2 = df.query('source == "Источник2"').is_send_complaint.sum()      #  число успехов Источник 1
n2 = df.query('source == "Источник2"').is_send_complaint.count()    #  число испытаний Источник 2
p1 = s1/n1               #  оценка вероятности успеха Источник 1
p2 = s2/n2               #  оценка вероятности успеха Источник 2
p = (s1 + s2)/(n1+n2)    #  оценка вероятности успеха оба источника
z = (p2-p1)/ ((p*(1-p)*((1/n1)+(1/n2)))**0.5)  #  z-метка

Найдем значение p-value с помощью z-теста

In [5]:
z1, p_value = sm.stats.proportions_ztest([s1, s2], [n1, n2])
p_value

0.09229131320879888

P-value (0.09) больше выбранного значения alpha (0.05). В таком случае мы не можем отклонить нулевую гипотезу. 

Проверим значение p-value методом хи-квадрат.

In [14]:
proportion.proportions_chisquare([s1, s2], [n1, n2])[1]

0.09229131320879909

Метод хи-квадрат подтвердил значение p-value.

Посчитаем статистическую мощность.

Сперва подготовим расчет effect_size.

In [15]:
effect_size = ((p1 - p2) ** 2 / p1) ** 0.5
effect_size

0.02091670015193822

In [8]:
power.GofChisquarePower().solve_power(effect_size=effect_size, nobs=min(n1,n2), alpha=0.05, n_bins=2)

0.4863843726519206

Мощность невысокая. Посчитаем размер выборки, необходимый для достижения статистической мощности 0.8.

In [12]:
power.GofChisquarePower().solve_power(effect_size=effect_size, nobs=None, alpha=0.05, power=0.8, n_bins=2)

17939.91039167791

### Вывод:
При выбраном значении alpha (0.05) мы не можем отклонить нулевую гипотезу о равенстве конверсий из двух источников. 

Также статистическая мощность получилась всего 0.49. 

Рекомендуется продолжить тест, пока минимальное количество наблюдений в каждой выборке не достигнет 17940. В таком случае удастся достичь статистической мощности 0.8. 